## Using Agents in LlamaIndex
### Environement Setup

In [1]:
!pip install -q -r requirements.txt

In [2]:
from my_config import MyConfig
my_config = MyConfig()

`LlamaIndex` supports three main types of reasoning agents:

1. `Function Calling Agents` - These work with AI models that can call specific functions.
2. `ReAct Agents` - These can work with any AI that does chat or text endpoint and deal with complex reasoning tasks.
3. `Advanced Custom Agents` - These use more complex methods to deal with more complex tasks and workflows.

### Basic Agents and States

#### Getting My LLM

In [3]:
from my_utils import RunPodLLamaAgentQwenLLM
model_id="Qwen/Qwen2.5-Coder-7B-Instruct"
vllm_api_base = f"https://{my_config.VLLM_LLM_INFERENCE_NODE_IP}-8000.proxy.runpod.net/v1/chat/completions"
llm = RunPodLLamaAgentQwenLLM(api_url=vllm_api_base, overriden_model_name=model_id)

/Users/potter/Documents/MLArena/MLOps-Platform-Project/Hands-On/hfagnt-2-2-the-llamaindex-framework/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Very Basic Workflow

In [4]:
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent
from llama_index.core.tools import FunctionTool

# define sample Tool -- type annotations, function names, and docstrings, are all included in parsed schemas!
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the resulting integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Adds two integers and returns the resulting integer"""
    return a + b

# initialize agent
agent = AgentWorkflow.from_tools_or_functions(
    [FunctionTool.from_defaults(multiply), FunctionTool.from_defaults(add)],
    llm=llm
)

# workflow = FunctionAgent(
#     agent=[multiply, add],
#     llm=llm,
#     system_prompt="You are an agent that can perform basic mathematical operations using tools.",
# )

**Agents are stateless by default**, add remembering past interactions is opt-in using a `Context` object This might be useful if you want to use an agent that needs to remember previous interactions, like a chatbot that maintains context across multiple messages or a task manager that needs to track progress over time.

In [5]:
# stateless
# response = await agent.run("What is 2 times 2?")
# response
response = await agent.run(user_msg="What is 20+(2*4)?")
print(response.response)

Model response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {"a": 2, "b": 4}
Model response: assistant: Thought: Now I have the result of the multiplication. I need to add 20 to it.
Action: add
Action Input: {'a': 20, 'b': 8}
Model response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 28
assistant: 28


In [6]:
# remembering state
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)

Model response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Answer: Hello, Bob! How can I assist you today?
Model response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Your name is Bob.


### Very Basic Workflow with Context

In [7]:
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import ReActAgent, AgentWorkflow

# Define some tools
async def add(ctx: Context, a: int, b: int) -> int:
    """Add two numbers."""
    # update our count
    cur_state = await ctx.get("state")
    cur_state["num_fn_calls"] += 1
    print(cur_state)
    await ctx.set("state", cur_state)
    return a + b

async def multiply(ctx: Context, a: int, b: int) -> int:
    """Multiply two numbers."""
    # update our count
    cur_state = await ctx.get("state")
    cur_state["num_fn_calls"] += 1
    print(cur_state)
    await ctx.set("state", cur_state)
    return a * b



# we can pass functions directly without FunctionTool -- the fn/docstring are parsed for the name/description
multiply_agent = ReActAgent(
    name="multiply_agent",
    description="Is able to multiply two integers",
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    tools=[multiply],
    llm=llm,
)

addition_agent = ReActAgent(
    name="add_agent",
    description="Is able to add two integers",
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools=[add],
    llm=llm,
)



workflow = AgentWorkflow(
    agents=[multiply_agent, addition_agent],
    root_agent="multiply_agent",
    initial_state={"num_fn_calls": 0},
    state_prompt="Current state: {state}. User message: {msg}",
)

In [9]:
# run the workflow with context
ctx = Context(workflow)
response = await workflow.run(user_msg="What is the output of (3*5)+(10*13)*10?", ctx=ctx)

# pull out and inspect the state
state = await ctx.get("state")
print(state["num_fn_calls"])

Model response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {"a": 3, "b": 5}
{'num_fn_calls': 5}
Model response: assistant: Thought: I need to use another multiplication operation before adding the result.
Action: multiply
Action Input: {'a': 10, 'b': 13}
{'num_fn_calls': 6}
Model response: assistant: Thought: Now I need to multiply the result by 10 and then add it to the previous result.
Action: multiply
Action Input: {'a': 130, 'b': 10}
{'num_fn_calls': 7}
Model response: assistant: Thought: Finally, I need to add the results of the first multiplication to the result of the last multiplication.
Action: multiply
Action Input: {'a': 15, 'b': 1300}
{'num_fn_calls': 8}
Model response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The output of (3*5)+(10*13)*10 is 19500.
8
